# Análisis EVI y NDVI

## Dependencias

In [248]:
import pandas as pd
import os
import datetime

## Lectura de los datos

Se leerán los datos de ambas métricas, para ambas localizaciones y para todas las especies. Estos datos se incluirán en un solo dataframe

In [249]:
root = 'Material Sara Dennis' # Ruta raíz de los datos
metricas = ['EVI', 'NDVI']
localizaciones = ['DENTRO', 'FUERA']

In [250]:
# Dataframe en donde se incluirán todos los datos
df = pd.DataFrame()

# Se iteran las métricas
for metrica in metricas:
    # Se iteran los lugares
    for localizacion in localizaciones:
        # Se crea el path de la carpeta desde donde se intentarán leer 
        # los archivos de datos
        path = f'{root}/{metrica}/{metrica}-{localizacion}'
        # Se itera la carpeta y se leen los conjuntos de datos
        for f in os.listdir(path):
            # Se revisa si el objeto actual es un archivo (para no intentar)
            # leer carpetas
            path_dataset = f'{path}/{f}'
            if os.path.isfile(path_dataset):
                # Se obtiene el nombre de la especie en cuestión
                especie = f.split('.')[0]
                # Se lee el conjunto de datos y se añaden las columnas de
                # 'especie' y 'localizacion'
                df_tmp = pd.read_csv(dataset)
                df_tmp['especie'] = especie
                df_tmp['localizacion'] = localizacion
                df_tmp['metrica'] = metrica
                # Se concatenan los datos actuales en el DataFrame total
                df = df.append(df_tmp)

In [251]:
df

,X,Y,18-feb.,5-mar.,21-mar.,6-abr.,22-abr.,8-may.,24-may.,9-jun.,...,29-sep..5,15-oct..5,31-oct..5,16-nov..5,2-dic..5,18-dic..5,01-ene.,especie,localizacion,metrica
0,692598.684,3249897.334,2689,3060,2819,2533,2776,2543,2564,2436,...,4200,4120,3855,3803,3692,3694,3666,ENCINAR,DENTRO,EVI
1,692818.967,3249897.334,2526,3487,3404,3053,2506,2966,2772,3001,...,4200,4120,3855,3803,3692,3694,3666,ENCINAR,DENTRO,EVI
2,692598.684,3249016.200,2715,2908,3022,2840,2583,2432,2648,4135,...,5183,4965,4652,4376,4452,4295,4439,ENCINAR,DENTRO,EVI
3,692818.967,3248795.917,2859,3225,2867,2764,2680,2564,2782,4305,...,5183,4965,4652,4376,4452,4295,4439,ENCINAR,DENTRO,EVI
4,693039.250,3248575.633,2713,2990,2867,2863,2419,2451,2576,4454,...,5161,4848,4500,4324,4298,4231,4221,ENCINAR,DENTRO,EVI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,692158.117,3226767.581,3634,3468,3272,3052,2734,2777,2629,3770,...,5376,5035,4767,4536,4437,4143,3930,VEGARRO,FUERA,NDVI
28,692598.684,3226547.297,3016,3122,3063,2891,2623,2623,2484,4249,...,4764,4213,3847,3671,3427,3652,3349,VEGARRO,FUERA,NDVI
29,693039.250,3225886.447,2842,3045,2847,2872,2566,2567,2461,2453,...,4568,4042,3743,3693,3587,3500,3228,VEGARRO,FUERA,NDVI
30,688853.867,3223022.764,3157,3803,3577,3371,3453,2926,3296,5553,...,4904,4776,4399,4151,4245,4250,4083,VEGARRO,FUERA,NDVI


##  Corrección del formato de las fechas

Se cambiará el formato de las fechas que aparece en el nombre de las columnas para utilizar el formato estándar de fechas. `(yyyy-mm-dd)` 

In [252]:
# Diccionario para mapear los meses de string a int
meses = {
    'ene':1,'feb':2,'mar':3,'abr':4,'may':5,'jun':6,
    'jul':7,'ago':8,'sep':9,'oct':10,'nov':11,'dic':12
}

In [253]:
# Año del inicio de las mediciones
anio = 2000
# Diccionario para almacenar las nuevas fechas
nuevas_cols = {}
# Bandera para indicar si ha comenzado un año nuevo
mes_anterior = 0
# Se iteran todas las columnas correspondientes a fechas
for fecha in df.columns[2:-3]:
    # Se obtiene el día y el mes
    dia = int(fecha.split('-')[0])
    mes = meses[fecha.split('.')[0].split('-')[1]]
    # Si el mes actual es enero, pero el mes anterior fue diciembre, entonces
    # acaba de iniciar un año nuevo y se actualiza
    if mes == 1 and mes_anterior == 12:
        anio += 1
    # Se actualiza la bandera del mes anterior
    mes_anterior = mes
    # La fecha tipo string se relaciona con la misma fecha pero en formato datetime
    nuevas_cols[fecha] = pd.to_datetime(datetime.datetime(anio, mes, dia).strftime('%Y-%m-%d'))

# Se renombran las columnas con las fechas en formato datetime
df = df.rename(columns=nuevas_cols)
df

,X,Y,2000-02-18 00:00:00,2000-03-05 00:00:00,2000-03-21 00:00:00,2000-04-06 00:00:00,2000-04-22 00:00:00,2000-05-08 00:00:00,2000-05-24 00:00:00,2000-06-09 00:00:00,...,2020-09-29 00:00:00,2020-10-15 00:00:00,2020-10-31 00:00:00,2020-11-16 00:00:00,2020-12-02 00:00:00,2020-12-18 00:00:00,2021-01-01 00:00:00,especie,localizacion,metrica
0,692598.684,3249897.334,2689,3060,2819,2533,2776,2543,2564,2436,...,4200,4120,3855,3803,3692,3694,3666,ENCINAR,DENTRO,EVI
1,692818.967,3249897.334,2526,3487,3404,3053,2506,2966,2772,3001,...,4200,4120,3855,3803,3692,3694,3666,ENCINAR,DENTRO,EVI
2,692598.684,3249016.200,2715,2908,3022,2840,2583,2432,2648,4135,...,5183,4965,4652,4376,4452,4295,4439,ENCINAR,DENTRO,EVI
3,692818.967,3248795.917,2859,3225,2867,2764,2680,2564,2782,4305,...,5183,4965,4652,4376,4452,4295,4439,ENCINAR,DENTRO,EVI
4,693039.250,3248575.633,2713,2990,2867,2863,2419,2451,2576,4454,...,5161,4848,4500,4324,4298,4231,4221,ENCINAR,DENTRO,EVI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,692158.117,3226767.581,3634,3468,3272,3052,2734,2777,2629,3770,...,5376,5035,4767,4536,4437,4143,3930,VEGARRO,FUERA,NDVI
28,692598.684,3226547.297,3016,3122,3063,2891,2623,2623,2484,4249,...,4764,4213,3847,3671,3427,3652,3349,VEGARRO,FUERA,NDVI
29,693039.250,3225886.447,2842,3045,2847,2872,2566,2567,2461,2453,...,4568,4042,3743,3693,3587,3500,3228,VEGARRO,FUERA,NDVI
30,688853.867,3223022.764,3157,3803,3577,3371,3453,2926,3296,5553,...,4904,4776,4399,4151,4245,4250,4083,VEGARRO,FUERA,NDVI


## 'Tidyficación' de los datos

Un conjunto **Tidy** de datos es aquel en el que cada renglón corresponde a una observación única y cada columna corresponde a una variable única. Como podemos ver en los datos originales, la fecha debería ser una variable; sin embargo, los valores de esta variable se encuentra disperso a lo largo de 486 columnas. A continuación se corregirá este detalle.

In [254]:
df = df.melt(
    id_vars=['X','Y','especie','localizacion','metrica'],
    value_vars=df.columns[2:-3],
    var_name='fecha',
    value_name='valor'
)
df.fecha = pd.to_datetime(df.fecha)
df = df[['X','Y','fecha','especie','localizacion','metrica','valor']]

In [255]:
df

,X,Y,fecha,especie,localizacion,metrica,valor
0,692598.684,3249897.334,2000-02-18,ENCINAR,DENTRO,EVI,2689
1,692818.967,3249897.334,2000-02-18,ENCINAR,DENTRO,EVI,2526
2,692598.684,3249016.200,2000-02-18,ENCINAR,DENTRO,EVI,2715
3,692818.967,3248795.917,2000-02-18,ENCINAR,DENTRO,EVI,2859
4,693039.250,3248575.633,2000-02-18,ENCINAR,DENTRO,EVI,2713
...,...,...,...,...,...,...,...
430971,692158.117,3226767.581,2021-01-01,VEGARRO,FUERA,NDVI,3930
430972,692598.684,3226547.297,2021-01-01,VEGARRO,FUERA,NDVI,3349
430973,693039.250,3225886.447,2021-01-01,VEGARRO,FUERA,NDVI,3228
430974,688853.867,3223022.764,2021-01-01,VEGARRO,FUERA,NDVI,4083
